<a href="https://colab.research.google.com/github/jacmal/RL-Crypto-Trading-AI-BOT/blob/main/ENV_new_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO !!!
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import copy
import pandas as pd
import numpy as np
import random
from collections import deque
#from tensorboardX import SummaryWriter
from tensorflow.keras.optimizers import Adam, RMSprop
#from model import Actor_Model, Critic_Model, Shared_Model
#from utils import TradingGraph, Write_to_file, Normalizing
import matplotlib.pyplot as plt
from datetime import datetime
#from indicators import *
#from multiprocessing_env import train_multiprocessing, test_multiprocessing
import json

import requests
import io

In [ ]:
# load data sets
url_btc = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BTCUSDT_1h.csv"
url_eth = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_ETHUSDT_1h.csv"
#url_bnb = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BNBUSDT_1h.csv"

site_btc = requests.get(url_btc).content
#site_eth = requests.get(url_eth).content

df_btc = pd.read_csv(io.StringIO(site_btc.decode('utf-8')))
#df_eth = pd.read_csv(io.StringIO(site_eth.decode('utf-8')))

# reverse data sets
df_btc = df_btc.loc[::-1].reset_index(drop=True)
df_btc = df_btc.iloc[1:,:]

#df_eth = df_eth.loc[::-1].reset_index(drop=True)
#df_eth = df_eth.iloc[1:,:]

#
col_to_drop = ['unix', 'symbol', 'Volume USDT', 'tradecount', 'Volume BTC']

df_btc.drop(labels=col_to_drop, axis=1, inplace=True)
#df_eth.drop(labels=col_to_drop, axis=1, inplace=True)

#df_btc.set_index('date', drop=True, inplace=True)
#df_eth.set_index('date', drop=True, inplace=True)

df_btc['Open_btc'] = df_btc['open'].copy()

df_btc.head(5)

In [ ]:
# TODO !!!
################################################################################
### MAIN SETTINGS ###
#####################

# main path
FOLDER_PATH = "/content/drive/MyDrive/Projects/RL_Trading_BOT/"

# agent parameters file names
PARAMS_FILE = 'Parameters.json' # model parameters file

MODELS_FOLDER_NAME = 'RL_Models_Training'
MODEL_BEST = 'RL_Model_Best'
MODEL_NAME = 'RL_TradingAgent'
#ACTOR_NAME = '_Actor.h5'
#CRITIC_NAME = '_Critic.h5'

# environment file names
ACTUAL_DATA_FILE = 'actual_df_data.csv'       # actual crypto data
ACTUAL_DATA_NORM_FILE = 'actual_df_norm.csv'  # actual crypto data normalized
# trade history
TRADING_HISTORY_FILE = 'trade_history.csv'

################################################################################
# AGENT MODEL SETTINGS #
########################
# Reinforcement Learining - Proximal Policy Optimization (PPO) Adventage Actor Critic Model

#ACTOR_NAME = '_Actor.h5'
#CRITIC_NAME = '_Critic.h5'
LOOKBACK_WINDOW_SIZE = 50
INITIAL_MONEY_BALANCE = 1000
# Actor Critic Neural Network Hyperparameters
LEARNING_RATE = 0.00005
EPOCHS_NUMBER = 1
BATCH_SIZE = 32
# Agent Hyperparameters
GAMMA = 0.99
LAMBDA = 0.95

################################################################################
# TRADING ENVIRONMENT SETTINGS #
################################

SITE_FEES = 0.001
NORMALIZE_VALUE = 1 # 1 if NOT normalize
HOLD_CRYPTO_FACTOR_REWARD = 0.9
HOLD_MONEY_FACTOR_REWARD = 0.7
BUY_CRYPTO_FACTOR_REWARD = 0.1
#crypto_list = ['btc', 'eth', 'bnb', ...]
CRYPTO_LIST = ['btc']
simple_actions = ['buy', 'hold', 'sell']

In [ ]:
# TODO
# create program main folder
file_path = os.path.join(FOLDER_PATH, 'RL_Trading_BOT')
if not os.path.exists(file_path):
    os.makedirs(file_path)

#model_path = os.path.join(file_path, 'Model')
#if not os.path.exists(model_path):
#    os.makedirs(model_path)

In [ ]:
# TODO
def build_action_dict(crypto_list, action_list):
    '''
    TODO
    Create action dictionary.
    Args:
        crypto_list (list): List of cryptocurrency names. "['btc', 'eth', ...]"
        action_list (list): List of simple action names.  "['buy', 'sell', 'hold']"
    
    Returns:
        dict: Return dict with action number as key and action name as value.
    '''
    new_crypto_actions = ['hold_money']
    action = ''
    for crypto in crypto_list:
        for s_action in simple_actions:
            action = crypto + '_' + s_action
            new_crypto_actions.append(action)
            action = ''
    
    actions_dict = {key: val for key, val in enumerate(new_crypto_actions)}
    #actions_dict[-1] = 'episode_start'
    
    return actions_dict

ad = build_action_dict(CRYPTO_LIST, simple_actions)
print(ad)

In [ ]:
# DEV DEV DEV
class TradingEnvironment:
    '''
    TODO
    '''
    def __init__(self, df, df_normalized, lookback_window_size=LOOKBACK_WINDOW_SIZE, initial_balance=INITIAL_MONEY_BALANCE, normalize_value=NORMALIZE_VALUE, action_dict=ad, fees=SITE_FEES):
        self.df = df                                          # dataframe with all trading data
        self.df_normalized = df_normalized                    # dataframe with normalized value
        self.column_names = list(df_normalized.columns)[1:]   # list of columns to create a part of environment states without Date column
        
        self.normalize_value = normalize_value
        self.all_df_steps = df.shape[0] - 1
        self.lookback_window_size = lookback_window_size
        
        self.initial_balance = initial_balance
        self.fees = fees
        self.cryptos_list = CRYPTO_LIST
        
        # Action number to action label dictionary
        self.action_to_label_dict = action_dict
        # Action label to action number dictionary
        self.label_to_action_dict = {k: v for (v, k) in action_dict.items()}
        # Enviroment action space
        self.action_space = np.array(list(self.action_to_label_dict.keys()))
        
        # training history
        self.df_data_training_history = pd.DataFrame()
        
        # Market history values for the last lookback_window_size prices
        self.market_history = deque(maxlen=self.lookback_window_size) # DEV
        self.df_market_history = pd.DataFrame()
        self.df_market_window = pd.DataFrame()
        
        # Orders history contains the balance, net_worth, crypto_bought, crypto_sold, crypto_held values for the last lookback_window_size steps
        self.orders_history = deque(maxlen=self.lookback_window_size) # DEV
        self.df_orders_history = pd.DataFrame()
        self.df_ordess_window = pd.DataFrame()

        # Orders variable dictionary
        self.orders_var_dict = None

################################################################################
# DATA #
########
    # TODO
    def add_new_data(self, data_point_df):
        '''
        Add a new data point into main DataFrame and normalized DataFrame.
        '''
        data_point_df_normalized = data_point_df / self.normalize_value

        new_df = pd.concat([self.df, data_point_df], axis=0, ignore_index=True)
        new_df.reset_index(inplace=True, drop=True)
        
        new_df_normalized = pd.concat([self.df, data_point_df_normalized], axis=0, ignore_index=True)
        new_df_normalized.reset_index(inplace=True, drop=True)

    # TODO
    def save_market_dfs(self):
        dataframes_dir_path = os.path.join(FOLDER_PATH, 'dataframes')
        if not os.path.exists(dataframes_dir_path):
            os.makedirs(dataframes_dir_path)
        df_path = os.path.join(dataframes_dir_path, 'market_history.csv')
        df_norm_path = os.path.join(dataframes_dir_path, 'market_history_norm.csv')
        pd.to_csv(df_path)
        pd.to_csv(df_norm_path)
    
    # TODO
    def reset_market_data_window(self, idx_start, idx_end, col_names, norm_data=True):
        if norm_data==True:
            return self.df_normalized.loc[idx_start:idx_end, col_names].copy()
        elif norm_data==False:
            return self.df.loc[idx_start:idx_end, col_names].copy()

    # TODO
    def reset_orders_data_window(self, norm_data=True):
        if norm_data==True:
            orders_df = pd.DataFrame(self.orders_var_dict, index=[self.current_step])
            for i in range(self.current_step+1, self.current_step + self.lookback_window_size, 1):
                df_temp = pd.concat([orders_df, pd.DataFrame(self.orders_var_dict, index=[i])], axis=0)
                orders_df = df_temp.copy()
            return orders_df
        else:
            

    # TODO
    def add_trade_history_to_df(self):
        '''
        TODO
        '''
        if len(list(self.all_trade_history.keys())) == 0:
            new_data = {k: [v] for (k, v) in self.trades[-1].items()}
            self.all_trade_history = pd.DataFrame(new_data)
        else:
            frame_1, frame_2 = self.all_trade_history.copy(), pd.DataFrame({k: [v] for (k, v) in self.trades[-1].items()}) 
            self.all_trade_history = pd.concat([frame_1, frame_2], axis=0, ignore_index=True)

    # TODO
    def save_trades_df(self):
        '''
        TODO
        '''
        pass

    # TODO
    def auto_save_trades_df(self):
        '''
        TODO
        '''
        pass

    # TODO
    def actual_date_and_time(self, training=False):
        '''
        TODO
        '''
        if training == False:
            return datetime.now().strftime("%Y-%m-%d %H:%M:%S").split()
        elif training == True:
            return self.df.loc[self.current_step, 'Date']

################################################################################
# ENV #
#######

    # TODO
    def init_orders_variables_dict(self):
        '''
        Initialize orders_variable_dict dictionary for orders history.
        '''
        orders_var_dict = {'_balance': self.initial_balance,
                           '_net_worth': self.initial_balance,
                           '_prev_net_worth': self.initial_balance}
        
        for c in self.cryptos_list:
            orders_var_dict['_'+ c +'_bought_price'] = 0
        
        for action in self.action_to_label_dict.values():
            orders_var_dict[action] = 0
        
        #print(orders_var_dict)
        return orders_var_dict

    # TODO
    def possible_actions(self, reset=False):
        '''
        TODO
        '''
        possible_actions = []
        if reset == True: ### coś chyba nie tak ? !!!
            actions_list = list(ad.values())
            #print(actions_list)
            possible_actions.append(self.label_to_action_dict['hold_money'])
            for action in actions_list:
                if action.split('_')[1] == 'buy':
                    possible_actions.append(self.label_to_action_dict[action])
            return possible_actions
        else:
            prev_action_name = self.action_to_label_dict[self.prev_action]
            crypto_name = prev_action_name.split('_')[0]
            prev_action_type = prev_action_name.split('_')[1]
            #print(self.prev_action)
            #print(prev_action_name, action_type)
            #print(input('BUG!'))
            if prev_action_type == 'hold':
                possible_actions.append(self.label_to_action_dict[crypto_name+'_hold'])
                possible_actions.append(self.label_to_action_dict[crypto_name+'_sell'])
                return possible_actions
            elif prev_action_type == 'buy':
                possible_actions.append(self.label_to_action_dict[crypto_name+'_hold'])
                return possible_actions
            elif prev_action_type == 'sell' or prev_action_name == 'hold_money':
                possible_actions.append(self.label_to_action_dict['hold_money'])
                for crypto in self.cryptos_list:
                    possible_actions.append(self.label_to_action_dict[crypto+'_buy'])
                return possible_actions

    # Get the data points for the given current_step
    # TODO
    def next_observation(self):
        '''
        TODO
        '''
        self.market_history.append([self.df_normalized.loc[self.current_step, column] for column in self.column_names])
        #print(self.orders_history)
        #print(np.array(self.orders_history))
        #print(np.array(self.orders_history).shape)
        #print(input())
        obs = np.concatenate((self.orders_history, self.market_history), axis=1)
        return obs

    # Calculate reward
    # TODO
    def get_reward(self, reward_type):
        '''
        TODO
        '''
        # TODO
        if self.episode_orders > 1 and self.episode_orders > self.prev_episode_orders:
            self.prev_episode_orders = self.episode_orders
            
            # TODO
            if reward_type == 'hold':
                reward = HOLD_CRYPTO_FACTOR_REWARD * (self.trades[-1]['total_crypto'] * self.trades[-1]['current_price'] - self.trades[-1]['bought_price'] * self.trades[-1]['total_crypto'])
                self.trades[-1]['Reward'] = reward
                return reward

            # TODO         
            elif reward_type == 'buy': # ??? coś chyba nie tak - sprawdzić - chyba OK wzrost wartości konta wzrost nagrody
                reward = BUY_CRYPTO_FACTOR_REWARD * (self.trades[-1]['net_worth'] - self.initial_balance) #/ self.initial_balance)
                self.trades[-1]['Reward'] = reward
                return reward
            
            # TODO
            elif reward_type == 'sell': # sprawdzić self.trades[-2] ???
                reward = self.trades[-2]['total_crypto'] * self.trades[-2]['current_price'] - self.trades[-2]['bought_price'] * self.trades[-2]['total_crypto']
                self.trades[-1]['Reward'] = reward
                return reward

            # TODO
            elif reward_type == 'money': # jak będzie wiecej krypto to będzie się zle nagroda liczyła i będzie trzeba zmienić na ceny średnie
                reward = HOLD_MONEY_FACTOR_REWARD * (self.trades[-1]['bought_price'] - self.trades[-1]['current_price']) * (self.trades[-1]['total_balance'] / self.trades[-1]['current_price'])
                self.trades[-1]['Reward'] = reward
                return reward
        else:
            return 0

    # TODO
    def reset_state(self, env_steps_size=0):        
        '''
        TODO
        '''
        self.orders_var_dict = self.init_orders_variables_dict()
        
        self.episode_orders = 0 # track episode orders count
        self.prev_episode_orders = 0 # track previous episode orders count

        self.env_steps_size = env_steps_size

        # DEV last trade and reward history
        #self.trades = deque(maxlen=100)
        #self.rewards = deque(maxlen=100)

        # 
        self.prev_action = 0 # reset episode starting action into "hold_money" action
        self.start_episode = True
        #self.possible_actions = self._possible_actions(reset=True)

        if env_steps_size > 0: # used for training dataset
            self.start_step = random.randint(self.lookback_window_size, self.all_df_steps - env_steps_size)
            self.end_step = self.start_step + env_steps_size
        else: # used for testing dataset
            # OLD
            self.start_step = self.lookback_window_size
            self.end_step = self.all_df_steps
            # DEV NEW
            
            
        self.current_step = self.start_step
        
        for i in reversed(range(self.lookback_window_size)):
            current_step = self.current_step - i
            
            orders_data = list(self.orders_var_dict.values())
            orders_normalized = [x / self.normalize_value for x in orders_data]
            self.orders_history.append(orders_normalized)
        
            # one line for loop to fill market history withing reset state values
            self.market_history.append([self.df_normalized.loc[current_step, column] for column in self.column_names])

        #test_l = []
        #for i in reversed(range(self.lookback_window_size)):
        #    current_step = self.current_step - i
        #    print(current_step)
        #    test_l.append(current_step)
        #print(test_l)

        #print(self.orders_history[-1])
        #print(self.market_history[-1])
        #state = np.concatenate((self.orders_history, self.market_history), axis=1)

        #return state

    # TODO
    def save_state(self, training=False, state_name='state_backup'):
        '''
        TODO
        '''
        if training==False:
            state = np.concatenate((self.orders_history, self.market_history), axis=1)
            np.save()
        elif training==True:
            pass

    # TODO
    def load_state(self, state_name):
        '''
        TODO
        '''
        pass

    # TODO
    def step(self, action, training=True):
        '''
        TODO
        '''
        self.current_step += 1
        
        action_name = self.action_to_label_dict[action]
        crypto_name = action_name.split('_')[0]
        action_type = action_name.split('_')[1]

        if self.start_episode == True:
            self.possible_actions = self._possible_actions(reset=True)
            self.start_episode = False
        elif self.start_episode == False:
            self.possible_actions = self._possible_actions()

        if action not in self.possible_actions:
            done = True
            obs = self._next_observation()
            reward = -10000
            return obs, reward, done

        self.prev_action = action

        # calculate market average crypto prices
        crypto_prices_sum = 0
        for crypto in crypto_list:
            crypto_prices_sum += self.df.loc[self.current_step, 'Open_' + crypto]
        crypto_prices_avg = crypto_prices_sum / len(crypto_list)
        
        if action_name != 'hold_money':
            crypto_price_col_name = 'Open_' + crypto_name
            #crypto_high_col_name = 'High_' + crypto_name
            #crypto_low_col_name = 'Low_' + crypto_name
            
            actual_df_price = self.df.loc[self.current_step, crypto_price_col_name]
            actual_df_date = self.df.loc[self.current_step, 'date']
            #actual_df_high = self.df.loc[self.current_step, crypto_high_col_name]
            #actual_df_low = self.df.loc[self.current_step, crypto_low_col_name]
        
        elif action_name == 'hold_money':
            actual_df_date = self.df.loc[self.current_step, 'date']
                    
        # hold any crypto actions
        if action_type == 'hold':
            self.crypto_bought_price = actual_df_price
            
            self.orders_var_dict['_balance'] = self.orders_var_dict['_balance']
            self.orders_var_dict['_prev_net_worth'] = self.orders_var_dict['_net_worth']
            self.orders_var_dict['_net_worth'] = self.orders_var_dict['_balance'] + self.orders_var_dict[crypto + '_buy'] * actual_df_price + self.orders_var_dict[crypto + '_hold'] * actual_df_price
            self.orders_var_dict['_crypto_bought_price'] = self.orders_var_dict['_crypto_bought_price']
            self.orders_var_dict['_market_avg_prices'] = crypto_prices_avg

            if self.orders_var_dict[crypto_name + '_buy'] != 0:
                self.orders_var_dict[action_name] = self.orders_var_dict[crypto_name + '_buy']
                self.orders_var_dict[crypto_name + '_buy'] = 0

            self.trades.append({'Date' : actual_df_date,
                                'total_crypto': self.orders_var_dict[action_name],
                                'total_balance': self.orders_var_dict['_balance'],
                                'net_worth': self.orders_var_dict['_net_worth'],
                                'prev_net_worth': self.orders_var_dict['_prev_net_worth'],
                                'action': action_name,
                                'type': action_type,
                                'crypto_name': crypto_name,
                                'current_price': actual_df_price,
                                'bought_price': self.orders_var_dict['_crypto_bought_price'],
                                'market_avg_prices': self.orders_var_dict['_market_avg_prices'],
                                'Reward': 0})
            
            self.episode_orders += 1
            
            reward = self._get_reward('hold')

        # buy any crypto actions
        elif action_type == 'buy' and self.orders_var_dict['_balance'] > self.initial_balance*0.05:
            self.crypto_bought = self.orders_var_dict['_balance'] / actual_df_price
            self.crypto_bought *= (1-self.fees) # substract fees
            self.crypto_bought_price = actual_df_price
            
            self.orders_var_dict['_balance'] -= self.crypto_bought * actual_df_price
            self.orders_var_dict['_prev_net_worth'] = self.orders_var_dict['_net_worth']
            self.orders_var_dict['_net_worth'] = self.orders_var_dict['_balance'] + self.crypto_bought * actual_df_price
            self.orders_var_dict['_crypto_bought_price'] = actual_df_price
            self.orders_var_dict['_market_avg_prices'] = crypto_prices_avg
            
            self.orders_var_dict[action_name] = self.crypto_bought
            self.orders_var_dict['hold_money'] = self.orders_var_dict['_balance']
                       
            self.trades.append({'Date': actual_df_date,
                                'total_crypto': self.orders_var_dict[action_name],
                                'total_balance': self.orders_var_dict['_balance'],
                                'net_worth': self.orders_var_dict['_net_worth'],
                                'prev_net_worth': self.orders_var_dict['_prev_net_worth'],
                                'action': action_name,
                                'type': action_type,
                                'crypto_name': crypto_name,
                                'current_price': actual_df_price,
                                'bought_price': actual_df_price,
                                'market_avg_prices': self.orders_var_dict['_market_avg_prices'],
                                'Reward': 0})

            self.episode_orders += 1

            reward = self._get_reward('buy')
        
        # sell any crypto actions
        elif action_type == 'sell' and self.orders_var_dict[crypto_name + '_hold'] * actual_df_price > self.initial_balance*0.05: ### !!! crypto held
            # Sell 100% of current crypto held
            self.crypto_sold = self.orders_var_dict[crypto_name + '_hold']
            self.crypto_sold *= (1-self.fees) # substract fees
            self.crypto_sold_price = actual_df_price
            self.crypto_bought_price = 0
            
            self.orders_var_dict['_balance'] += self.crypto_sold * actual_df_price
            self.orders_var_dict['_prev_net_worth'] = self.orders_var_dict['_net_worth']
            
            self.orders_var_dict[crypto_name + '_hold'] -= self.crypto_sold
            
            self.orders_var_dict['_net_worth'] = self.orders_var_dict['_balance'] + self.orders_var_dict[crypto_name + '_hold'] * actual_df_price
            self.orders_var_dict['_crypto_bought_price'] = actual_df_price
            self.orders_var_dict['_market_avg_prices'] = crypto_prices_avg          
            
            self.trades.append({'Date': actual_df_date,
                                'total_crypto': self.orders_var_dict[action_name],
                                'total_balance': self.orders_var_dict['_balance'],
                                'net_worth': self.orders_var_dict['_net_worth'],
                                'prev_net_worth': self.orders_var_dict['_prev_net_worth'],
                                'action': action_name,
                                'type': action_type,
                                'crypto_name': crypto_name,
                                'current_price': actual_df_price,
                                'bought_price': actual_df_price,
                                'market_avg_prices': self.orders_var_dict['_market_avg_prices'],
                                'Reward': 0})
            
            self.episode_orders += 1

            reward = self._get_reward('sell')

        elif action_type == 'money':          
            
            self.orders_var_dict['_crypto_bought_price'] = self.orders_var_dict['_crypto_bought_price']
            self.orders_var_dict['_market_avg_prices'] = crypto_prices_avg
            self.orders_var_dict['_prev_net_worth'] = self.orders_var_dict['_net_worth']
            self.orders_var_dict['_net_worth'] = self.orders_var_dict['_balance'] # coś tu do poprawy jest !!!

            self.orders_var_dict['hold_money'] = self.orders_var_dict['_balance']

            self.trades.append({'Date': actual_df_date,
                                'total_crypto': self.orders_var_dict[action_name],
                                'total_balance': self.orders_var_dict['_balance'],
                                'net_worth': self.orders_var_dict['_net_worth'],
                                'prev_net_worth': self.orders_var_dict['_prev_net_worth'],
                                'action': action_name,
                                'type': action_type,
                                'crypto_name': crypto_name,
                                'current_price': crypto_prices_avg,
                                'bought_price': self.orders_var_dict['_crypto_bought_price'],
                                'market_avg_prices': self.orders_var_dict['_market_avg_prices'],
                                'Reward': 0})
            
            self.episode_orders += 1

            reward = self._get_reward('money')

        orders_data = list(self.orders_var_dict.values())
        orders_normalized = [x / self.normalize_value for x in orders_data]
        self.orders_history.append(orders_normalized)
        #print('orders_history after append: ', self.orders_history)
        
        # add new trade to trades history dataframe
        self._add_trade_history_to_dataframe()

        obs = self._next_observation()
        
        if training==True:
            if self.orders_var_dict['_net_worth'] <= self.initial_balance/2:
                done = True
            else:
                done = False
        else:
            print('save all data')

        #obs = self._next_observation()
        
        return obs, reward, done